In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install seqeval

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
import csv
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
import ast

from datasets import Dataset

In [ ]:
train = pd.read_csv('http://sybmo.com/thesis/train.csv')
train.tokens = train.tokens.apply(ast.literal_eval)
train.labels = train.labels.apply(ast.literal_eval)

test = pd.read_csv('http://sybmo.com/thesis/train.csv')
test.tokens = test.tokens.apply(ast.literal_eval)
test.labels = test.labels.apply(ast.literal_eval)

test.labels = test.labels.astype(str).str.replace("2", "1").apply(ast.literal_eval)
train.labels = train.labels.astype(str).str.replace("2", "1").apply(ast.literal_eval)

train.head(5)

,id,tokens,labels,Sentence Label + Target Group
0,13543989_gab,"[gotta, keep, some, beaners, in, texas, texmex...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]",hatespeech targeting Hispanic
1,21496378_gab,"[you, deserve, death, if, you, voted, ted, cru...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]",hatespeech targeting Hispanic
2,15459655_gab,"[according, to, news, reports, the, jury, also...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",hatespeech targeting Hispanic
3,7164097_gab,"[he, has, not, done, anything, he, deported, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",hatespeech targeting Hispanic
4,18811707_gab,"[the, wall, is, not, the, final, solution, the...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",hatespeech targeting Hispanic


In [ ]:
label_list = [0,1]
label_encoding_dict = {0:0, 1:1}

task = "ner"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)



  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,
                                                        num_labels=len(label_list))

from transformers import EarlyStoppingCallback
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    "distilBERT-binary",
    evaluation_strategy = "epoch", #instead of step
    #eval_steps = 100,
    #save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    #learning_rate=0.01,     #default
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    num_train_epochs=3, #instead of eleven
    weight_decay=0.01,
    #metric_for_best_model = 'f1',
    #load_best_model_at_end=True,
    push_to_hub=True
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.132898,0.000000,0.000000,0.000000,0.948009
2,No log,0.108121,0.000000,0.000000,0.000000,0.959432
3,0.154400,0.097859,0.000000,0.000000,0.000000,0.963509


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: Sentence Label + Target Group, id, tokens. If Sentence Label + Target Group, id, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2784
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

TrainOutput(global_step=522, training_loss=0.15277562515945728, metrics={'train_runtime': 87.0644, 'train_samples_per_second': 95.929, 'train_steps_per_second': 5.996, 'total_flos': 120000789470016.0, 'train_loss': 0.15277562515945728, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to distilBERT-binary
Configuration saved in distilBERT-binary/config.json
Model weights saved in distilBERT-binary/pytorch_model.bin
tokenizer config file saved in distilBERT-binary/tokenizer_config.json
Special tokens file saved in distilBERT-binary/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/253M [00:00<?, ?B/s]

Upload file runs/Nov17_13-04-24_d0d7e0ee739e/events.out.tfevents.1668690274.d0d7e0ee739e.78.0:  62%|######2   …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/troesy/distilBERT-binary
   637c15e..ceef29b  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/troesy/distilBERT-binary
   637c15e..ceef29b  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.0}, {'name': 'Recall', 'type': 'recall', 'value': 0.0}, {'name': 'F1', 'type': 'f1', 'value': 0.0}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9635092367704423}]}
To https://huggingface.co/troesy/distilBERT-binary
   ceef29b..b099a9a  main -> main

   ceef29b..b099a9a  main -> main



'https://huggingface.co/troesy/distilBERT-binary/commit/ceef29bf9b1b2ca5ef295e9cde7d11ae8fe0afc6'

In [ ]:
predictions = []
model_name = "distilBERT-binary"
tokenizer = AutoTokenizer.from_pretrained(model_name)


censored_list = ['<sad>', '<laugh>', '<user>', '<censored>', '<number>', '<time>',
                 '<money>', '<percent>','k*ntucky','g.o.a.t.',';>>','r.i.','c*unt','l.a.']
tokenizer.add_tokens(censored_list)


model = AutoModelForTokenClassification.from_pretrained(model_name)

model.resize_token_embeddings(len(tokenizer))


for index, row in train.iterrows():
  words = row['tokens']
  sentence = ''
  for word in words:
    sentence = sentence + ' ' + word
  sentence = sentence.strip()
  encoding = tokenizer(sentence, return_tensors="pt")
  outputs = model(**encoding)
  logits = outputs.logits
  predicted_label_classes = logits.argmax(-1)
  predicted_labels = [model.config.id2label[id] for id in predicted_label_classes.squeeze().tolist()]
  sentence_predictions =[]
  for id, label in zip(encoding.input_ids.squeeze().tolist(), predicted_labels):
    t = tokenizer.decode([id]), label
    sentence_predictions.append(t)
  predictions.append(sentence_predictions)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file distilBERT-binary/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilBERT-binary",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file distilBERT-binary/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForTokenClassification.

All the weights of DistilBertForTokenClassification 

In [ ]:
with open("/drive/My Drive/thesis_sybren/BERTs/predictions/distil-binary.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(predictions)

FileNotFoundError: ignored

In [ ]:
from csv import reader
import re

final_list_words = []
final_list_labels = []

with open("/drive/My Drive/thesis_sybren/BERTs/predictions/distil-fresh-addedvoca.csv", "r") as read_obj:
    csv_reader = reader(read_obj)
    # iterate over each row in the csv using reader object
    breaki = 0
    for row in csv_reader:
        # declare emtpy lists for every row
        words = []
        labels = []
        # row variable is a list that represents a row in csv
        for element in row:
            # turn string representation of tuple into real tuple
            element = ast.literal_eval(str(element))
            # append every first element of tuple to words list
            words.append(element[0])

            # check if words start with bracket or double hashtag in order to only keep the right labels
            if not (element[0].startswith("[C") or element[0].startswith("##") or element[0].startswith("[S") ):
                labels.append(element[1])

        #remove "tokens" that start with bracket ([CLS] and [SEP] (but keep [UNK]))
        text = ' '.join([x for x in words if not (x.startswith('[C') or x.startswith('[S'))])
        clean_text = text.replace(' ##', '')


        split_fine_text = clean_text.split()

preds = [item for sublist in final_list_labels for item in sublist]
gold = [item for sublist in test.labels.to_list() for item in sublist]

In [ ]:
test_list = test.tokens.tolist()
for sent_index, sentence in enumerate(test_list):
    for token_index, token in enumerate(sentence):
        print(token.encode('utf8'))#.startswith(b'\\xf'):


In [ ]:
print(len(preds), print(len(gold)))

56710
0 None


In [ ]:
split_fine_text

['this',
 'should',
 'not',
 'be',
 'called',
 'or',
 'even',
 'considered',
 'womensmarch',
 'real',
 'women',
 'do',
 'not',
 'lend',
 'themselves',
 'or',
 'rely',
 'on',
 'this',
 'new',
 'feminist',
 'bullshit',
 'its',
 'superflous',
 'judgejudy',
 'said',
 'it',
 'best',
 'when',
 'she',
 'owned',
 'katie',
 'couric',
 'these',
 'girls',
 'have',
 'no',
 'love',
 'let',
 'alone',
 'respect',
 'for',
 'themselves',
 'in',
 'return',
 'they',
 'recieve',
 'none',
 'slagfest']